# LlamaIndex MCP Client Demo

This notebook demonstrates how to use the **LlamaIndex MCP implementation** (`BasicMCPClient` + `McpToolSpec`) to interact with the running NSO MCP server.

## Prerequisites
- LlamaIndex NSO MCP Server is running
- Virtual environment is activated
- All required packages are installed

## ⚠️ Important Note About Validation Errors

**Known Issue**: The LlamaIndex MCP client shows validation errors when calling tools. This is a **fundamental compatibility issue** between the MCP server's `CallToolResult` format and what the LlamaIndex MCP client expects.

**✅ What Works**:
- Connection to MCP server
- Tool discovery and listing
- Server-side tool execution (NSO functions work correctly)

**❌ What Doesn't Work**:
- Client-side validation of tool responses
- Clean error-free tool calling
- Production use with MCP

**🔧 Recommended Solution**: For production use, use the **pure LlamaIndex approach** without MCP (see `pure_llama_nso_agent.py`) which works perfectly without validation issues.

**📝 This Demo**: Shows the MCP approach for educational purposes, but expect validation errors.


## 1. Setup and Imports


In [30]:
# Install required packages if not already installed
import subprocess
import sys

def install_package(package):
    try:
        __import__(package)
        print(f"✅ {package} is already installed")
    except ImportError:
        print(f"📦 Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Install required packages
packages = [
    "mcp",
    "llama-index-tools-mcp",
    "llama-index-core",
    "llama-index-llms-azure-openai",
    "python-dotenv",
    "nest-asyncio"
]

for package in packages:
    install_package(package)


✅ mcp is already installed
📦 Installing llama-index-tools-mcp...



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


📦 Installing llama-index-core...



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


📦 Installing llama-index-llms-azure-openai...



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


📦 Installing python-dotenv...
📦 Installing nest-asyncio...



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [31]:
# Import required libraries
import asyncio
import logging
import os
import sys
from typing import Any, Dict, List

# LlamaIndex MCP imports
from llama_index.tools.mcp import BasicMCPClient, McpToolSpec

# LlamaIndex imports
from llama_index.core.tools import FunctionTool
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.core import Settings

# Environment setup
from dotenv import load_dotenv
import nest_asyncio

# Load environment variables
load_dotenv()

# Enable nested asyncio for Jupyter
nest_asyncio.apply()

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✅ All imports successful!")


✅ All imports successful!


## 2. Environment Configuration


In [32]:
# NSO Configuration
NSO_DIR = "/Users/gudeng/NCS-614"
os.environ['NCS_DIR'] = NSO_DIR
os.environ['DYLD_LIBRARY_PATH'] = f'{NSO_DIR}/lib'
os.environ['PYTHONPATH'] = f'{NSO_DIR}/src/ncs/pyapi'

# Add NSO Python API to Python path
nso_pyapi_path = f'{NSO_DIR}/src/ncs/pyapi'
if nso_pyapi_path not in sys.path:
    sys.path.insert(0, nso_pyapi_path)

print(f"✅ NSO environment configured: {NSO_DIR}")
print(f"✅ Python path updated with NSO API")


✅ NSO environment configured: /Users/gudeng/NCS-614
✅ Python path updated with NSO API


## 3. Create MCP Client and Test Connection


In [34]:
# Create LlamaIndex MCP Client
async def create_mcp_client():
    """Create and connect to MCP server using LlamaIndex BasicMCPClient."""
    try:
        print("🔧 Creating LlamaIndex MCP Client...")
        
        # Create BasicMCPClient for local process
        mcp_client = BasicMCPClient(
            "/Users/gudeng/MCP_Server/src/mcp_server/working/llama_index_mcp/start_fastmcp_nso_server.sh",
            args=[]
        )
        
        # Create McpToolSpec
        mcp_tool_spec = McpToolSpec(client=mcp_client)
        
        # Get tools from the server
        tools = await mcp_tool_spec.to_tool_list_async()
        
        print(f"✅ Connected to MCP server!")
        print(f"✅ Found {len(tools)} tools")
        
        return mcp_client, mcp_tool_spec, tools
        
    except Exception as e:
        print(f"❌ Failed to create MCP client: {e}")
        import traceback
        traceback.print_exc()
        return None, None, []

# Create the MCP client
mcp_client, mcp_tool_spec, tools = await create_mcp_client()


🔧 Creating LlamaIndex MCP Client...
✅ Connected to MCP server!
✅ Found 4 tools


## 4. List Available Tools


In [35]:
# List all available tools
if tools:
    print("📋 Available Tools:")
    for tool in tools:
        print(f"  • {tool.metadata.name}: {tool.metadata.description}")
        print(f"    Schema: {tool.metadata.fn_schema}")
        print()
else:
    print("❌ No tools available")


📋 Available Tools:
  • show_all_devices: Find out all available routers in the lab, return their names.
    Schema: <class 'llama_index.tools.mcp.base.show_all_devices_Schema'>

  • get_router_interfaces_config: Return configured interfaces (Loopback/GigabitEthernet/Ethernet) with IPv4 for a router.
    Schema: <class 'llama_index.tools.mcp.base.get_router_interfaces_config_Schema'>

  • configure_router_interface: Configure a router interface with IP address, description, and shutdown status.
    Schema: <class 'llama_index.tools.mcp.base.configure_router_interface_Schema'>

  • echo_text: Echo back the provided text (debug/health).
    Schema: <class 'llama_index.tools.mcp.base.echo_text_Schema'>



## 5. Test Individual Tools (With Expected Validation Errors)


In [36]:
# Test echo_text tool
async def test_echo_tool():
    """Test the echo_text tool."""
    if not tools:
        print("❌ No tools available")
        return None
    
    try:
        # Find echo_text tool
        echo_tool = None
        for tool in tools:
            if tool.metadata.name == "echo_text":
                echo_tool = tool
                break
        
        if not echo_tool:
            print("❌ echo_text tool not found")
            return None
        
        print("🔧 Testing echo_text tool...")
        print("⚠️  Note: You may see validation errors below - this is expected!")
        result = await echo_tool.acall(text="Hello from Jupyter Notebook!")
        print(f"Result: {result}")
        return result
        
    except Exception as e:
        print(f"❌ Error testing echo_text: {e}")
        import traceback
        traceback.print_exc()
        return None

# Test echo tool
echo_result = await test_echo_tool()


🔧 Testing echo_text tool...
⚠️  Note: You may see validation errors below - this is expected!
Result: meta=None content=[TextContent(type='text', text='Echo: Hello from Jupyter Notebook!', annotations=None, meta=None)] structuredContent={'result': 'Echo: Hello from Jupyter Notebook!'} isError=False


In [37]:
# Test show_all_devices tool
async def test_devices_tool():
    """Test the show_all_devices tool."""
    if not tools:
        print("❌ No tools available")
        return None
    
    try:
        # Find show_all_devices tool
        devices_tool = None
        for tool in tools:
            if tool.metadata.name == "show_all_devices":
                devices_tool = tool
                break
        
        if not devices_tool:
            print("❌ show_all_devices tool not found")
            return None
        
        print("🔧 Testing show_all_devices tool...")
        print("⚠️  Note: You may see validation errors below - this is expected!")
        result = await devices_tool.acall()
        print(f"Result: {result}")
        return result
        
    except Exception as e:
        print(f"❌ Error testing show_all_devices: {e}")
        import traceback
        traceback.print_exc()
        return None

# Test devices tool
devices_result = await test_devices_tool()


🔧 Testing show_all_devices tool...
⚠️  Note: You may see validation errors below - this is expected!
Result: meta=None content=[TextContent(type='text', text='Available devices: xr9kv-1, xr9kv-2, xr9kv-3', annotations=None, meta=None)] structuredContent={'result': 'Available devices: xr9kv-1, xr9kv-2, xr9kv-3'} isError=False


In [38]:
# Test get_router_interfaces_config tool
async def test_interfaces_tool():
    """Test the get_router_interfaces_config tool."""
    if not tools:
        print("❌ No tools available")
        return None
    
    try:
        # Find get_router_interfaces_config tool
        interfaces_tool = None
        for tool in tools:
            if tool.metadata.name == "get_router_interfaces_config":
                interfaces_tool = tool
                break
        
        if not interfaces_tool:
            print("❌ get_router_interfaces_config tool not found")
            return None
        
        print("🔧 Testing get_router_interfaces_config tool...")
        print("⚠️  Note: You may see validation errors below - this is expected!")
        result = await interfaces_tool.acall(router_name="xr9kv-3")
        print(f"Result: {result}")
        return result
        
    except Exception as e:
        print(f"❌ Error testing get_router_interfaces_config: {e}")
        import traceback
        traceback.print_exc()
        return None

# Test interfaces tool
interfaces_result = await test_interfaces_tool()


🔧 Testing get_router_interfaces_config tool...
⚠️  Note: You may see validation errors below - this is expected!
Result: meta=None content=[TextContent(type='text', text='Interfaces for xr9kv-3:\n  GigabitEthernet/0/0/0/0: No IP configured\n  GigabitEthernet/0/0/0/1: No IP configured\n  GigabitEthernet/0/0/0/2: No IP configured\n  Loopback/100: No IP configured', annotations=None, meta=None)] structuredContent={'result': 'Interfaces for xr9kv-3:\n  GigabitEthernet/0/0/0/0: No IP configured\n  GigabitEthernet/0/0/0/1: No IP configured\n  GigabitEthernet/0/0/0/2: No IP configured\n  Loopback/100: No IP configured'} isError=False


## 6. Summary and Results


## 7. Using Tools with LlamaIndex Agent (Recommended Approach)


In [39]:
# Create LlamaIndex Agent with MCP Tools
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.llms.azure_openai import AzureOpenAI
import requests
import base64
import json

# Get Azure OpenAI token (same as Flask app)
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
token_url = os.getenv("TOKEN_URL")
llm_endpoint = os.getenv("LLM_ENDPOINT")
appkey = os.getenv("APP_KEY")

# Create Basic auth header (like Flask app)
auth_string = f"{client_id}:{client_secret}"
auth_key = base64.b64encode(auth_string.encode()).decode()

headers = {
    "Accept": "*/*",
    "Content-Type": "application/x-www-form-urlencoded",
    "Authorization": f"Basic {auth_key}",
}

token_response = requests.post(token_url, headers=headers, data="grant_type=client_credentials")
token = token_response.json().get("access_token")

# Create user parameter for additional_kwargs
user_param = json.dumps({"appkey": appkey})

# Initialize Azure OpenAI LLM (Fixed configuration matching Flask app)
llm = AzureOpenAI(
    azure_endpoint=llm_endpoint,
    api_version="2024-07-01-preview",
    deployment_name="gpt-4o-mini",
    api_key=token,
    max_tokens=3000,
    temperature=0.1,
    additional_kwargs={"user": user_param}
)

# Create agent with MCP tools
if tools:
    print("🤖 Creating LlamaIndex FunctionAgent with MCP tools...")
    agent = FunctionAgent(
        tools=tools,
        llm=llm,
        system_prompt="""You are a network automation assistant. When users ask about:
- Device lists: Use show_all_devices tool
- Interface configurations: Use get_router_interfaces_config tool with the specific router name
- Interface configuration changes: Use configure_router_interface tool with router_name, interface_name, and optional parameters (ip_address, description, shutdown)
- Router 3 means xr9kv-3
- Router 1 means xr9kv-1  
- Router 2 means xr9kv-2

IMPORTANT: Interface names must use the format "Type/Number" (e.g., "Loopback/100", "GigabitEthernet/0/0/0/0").
When users say "Loopback 100" or "Loopback100", convert it to "Loopback/100".
When users say "GigabitEthernet 0/0/0/0", convert it to "GigabitEthernet/0/0/0/0".

Always use the appropriate tool to get the requested information or make configuration changes."""
    )
    print("✅ FunctionAgent created successfully!")
    print(f"✅ LLM configured: {llm.model}")
    print(f"✅ Azure Deployment: {llm.azure_deployment}")
    print(f"✅ Endpoint: {llm.azure_endpoint}")
else:
    print("❌ No tools available to create agent")
    agent = None


🤖 Creating LlamaIndex FunctionAgent with MCP tools...
✅ FunctionAgent created successfully!
✅ LLM configured: gpt-35-turbo
✅ Azure Deployment: None
✅ Endpoint: https://chat-ai.cisco.com


In [40]:
# Test the agent with your question: "show me all devices"
if agent:
    print("🔧 Testing agent with question:'")
    print("=" * 60)
    
    try:
        # Ask the agent to show all devices (note: await is needed for FunctionAgent)
        response = await agent.run("'can you get_router_interfaces_config for router xr9kv-3 ")
        print(f"\n🤖 Agent Response:")
        print(response)
        
    except Exception as e:
        print(f"❌ Error with agent: {e}")
        import traceback
        traceback.print_exc()
else:
    print("❌ No agent available")


🔧 Testing agent with question:'


INFO:httpx:HTTP Request: POST https://chat-ai.cisco.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-07-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://chat-ai.cisco.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-07-01-preview "HTTP/1.1 200 OK"



🤖 Agent Response:
Here are the interface configurations for router xr9kv-3:

- **GigabitEthernet/0/0/0/0**: No IP configured
- **GigabitEthernet/0/0/0/1**: No IP configured
- **GigabitEthernet/0/0/0/2**: No IP configured
- **Loopback/100**: No IP configured


In [42]:
# Test the agent with your question: "show me all devices"
if agent:
    print("🔧 Testing agent with question:'")
    print("=" * 60)
    
    try:
        # Ask the agent to show all devices (note: await is needed for FunctionAgent)
        response = await agent.run("'can you configure interface Loopback 101 on all routers with ipv4 address 2.1.1.x x is basically the routers number ")
        print(f"\n🤖 Agent Response:")
        print(response)
        
    except Exception as e:
        print(f"❌ Error with agent: {e}")
        import traceback
        traceback.print_exc()
else:
    print("❌ No agent available")


🔧 Testing agent with question:'


INFO:httpx:HTTP Request: POST https://chat-ai.cisco.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-07-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://chat-ai.cisco.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-07-01-preview "HTTP/1.1 200 OK"



🤖 Agent Response:
The interface Loopback/101 has been successfully configured on all routers with the following IPv4 addresses:

- **xr9kv-1**: IP Address 2.1.1.1
- **xr9kv-2**: IP Address 2.1.1.2
- **xr9kv-3**: IP Address 2.1.1.3


## 8. Interactive Testing - Ask Your Own Questions
